In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib widget
from joblib import Parallel, delayed
from scipy.optimize import curve_fit
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern",
    "figure.dpi": 300,
})

from Finite_volume_method_1D import *

In [ ]:
chippath = '../../DataKian/bTa_LT278W2chip5/'
filename = 'KID3_96dBm__TmK20_avgpulse_ampphase.csv'
length = 90

lambdanm = 402
KID402 = KID_data(chippath,lambdanm,filename,length)
KID402.fit_tail(start=250,end=-1)

In [ ]:
params = KID_params(eta_pb=0.57,sigma_IC=1,Teff=0.11,KID=KID402,trickle_time=0.03)
dtset=0.001
dxfrac=1/4
simtime=50
SIM = KID_sim(params=params,dt_init=dtset,dx_or_fraction=1/8,D_const=False,simtime_approx=simtime)

In [ ]:
params = KID_params(eta_pb=0.57,sigma_IC=1,Teff=0.11,KID=KID402)
dtset=0.001
dxfrac=1/4
simtime=50

SIMIC = KID_sim(params=params,dt_init=dtset,dx_or_fraction=1/8,D_const=False,simtime_approx=simtime)

In [ ]:
alpha=0.5
plt.close()
plt.figure()
plt.plot(SIM.t_axis_interp-SIM.t_start,SIM.phaseintime,color='r',alpha=alpha,label=r'Forcing term, $\tau=1\mu s$')
plt.plot(SIM.t_axis,SIM.Nqpintime*SIM.params.dthetadN,color='r',ls='--',alpha=alpha)
plt.plot(SIMIC.t_axis_interp-SIMIC.t_start,SIMIC.phaseintime,color='b',alpha=alpha,label='Initial condition')
plt.plot(SIMIC.t_axis,SIMIC.Nqpintime*SIMIC.params.dthetadN,color='b',ls='--',alpha=alpha)
plt.semilogy()
plt.xlabel(r'time ($\mu s$)')
plt.ylabel(r'phase (rad)')
plt.legend()
plt.show()

In [ ]:
freq = 20 #hz
fig, ax=plt.subplots()
ax.axis([-5,5,0,np.max([np.max(Q) for Q in SIM.Qintime])])
line, = ax.plot([],[])

def animate(i):
    xpos = SIM.x_centers_list[i]
    xneg = -np.flip(xpos)
    x = np.concatenate([xneg,xpos])
    Qpos = SIM.Qintime[i]
    Qneg = np.flip(Qpos)
    Q=np.concatenate([Qneg,Qpos])
    line.set_data(x,Q)
    ax.set_title("time (us): {:.4f}, frame: {}".format(SIM.t_axis[i],i))

ani = animation.FuncAnimation(fig, animate, frames=500, interval=1000/freq, blit=True)

ani.save('movie.mp4',fps=freq,dpi=200)

In [ ]:
SIM.trickle_total

In [ ]:
SIM.Nqp_init

In [ ]:
plt.figure()
plt.plot(SIM.t_axis)
plt.show()

plt.figure()
plt.plot(SIM.dtlist)
plt.show()